In [ ]:
%run buffered_leave_one_out.py -v --nargs 1 --experiment 15VEG_FAPAR local

In [ ]:
from wildfires.analysis import cube_plotting

master_mask = np.zeros((720, 1440), dtype=np.bool_)
lat_inds = np.arange(720).reshape(-1, 1)
lon_inds = np.arange(1440).reshape(1, -1)

vis_rad = 10

for result in results:
    master_mask[
        (((lat_inds - result[0][0][0]) ** 2 + (lon_inds - result[0][0][1]) ** 2) ** 0.5)
        < vis_rad
    ] = True

fig = cube_plotting(master_mask)

In [ ]:
from collections import defaultdict
from operator import itemgetter

from sklearn.metrics import r2_score

In [ ]:
len(args[0]) / len(experiments)

In [ ]:
len(experiments)

In [ ]:
data = defaultdict(list)
# assert len(results) == len(args[0])

n_seed_bins = 3
seed_bins = np.linspace(0, 1000, n_seed_bins + 1, dtype=np.int64)

for i in range(len(results)):
    experiment, radius, max_rad, seed = map(itemgetter(i % 8000), args)
    (
        (test_indices, n_ignored, n_train, n_hold_out, total_samples),
        test_y,
        pred_y,
    ) = results[i]

    seed_index = np.digitize(seed, seed_bins)

    data[(radius, "n_ignored")].append(n_ignored)
    data[(radius, "n_train")].append(n_train)
    data[(radius, "n_hold_out")].append(n_hold_out)
    data[(radius, "total_samples")].append(total_samples)
    data[(seed_index, radius, "test_y")].append(np.array(test_y).ravel())
    data[(seed_index, radius, "pred_y")].append(np.array(pred_y).ravel())

In [ ]:
keys = tuple(k for k in data if len(k) == 3)
seed_indices = tuple(sorted(set(map(itemgetter(0), keys))))
radii = tuple(sorted(set(map(itemgetter(1), keys))))
seed_indices, radii

In [ ]:
seed_r2s = [[] for _ in seed_indices]
for seed_i, seed_index in enumerate(seed_indices):
    for i, radius in enumerate(radii):
        test_y = np.array(data[(seed_index, radii[i], "test_y")]).ravel()
        pred_y = np.array(data[(seed_index, radii[i], "pred_y")]).ravel()
        seed_r2s[seed_i].append(r2_score(y_true=test_y, y_pred=pred_y))

plt.figure(dpi=200)
plt.title(experiment.name)
for seed_index, r2s in zip(seed_indices, seed_r2s):
    plt.plot(np.array(radii) * 30, r2s, label=f"Seed bin: {seed_index}")

plt.legend()
plt.xlabel("radius (approx. km)")
_ = plt.ylabel(r"CV $\mathrm{R}^2$")

In [ ]:
seed_r2s = [[] for _ in seed_indices]
for seed_i, seed_index in enumerate(seed_indices):
    for i, radius in enumerate(radii):
        test_y = np.array(data[(seed_index, radii[i], "test_y")]).ravel()
        pred_y = np.array(data[(seed_index, radii[i], "pred_y")]).ravel()
        seed_r2s[seed_i].append(r2_score(y_true=test_y, y_pred=pred_y))

plt.figure(dpi=200)
plt.title(experiment.name)
for seed_index, r2s in zip(seed_indices, seed_r2s):
    plt.plot(np.array(radii), r2s, label=f"Seed bin: {seed_index}")

plt.legend()
plt.xlabel("radius (pixels)")
_ = plt.ylabel(r"CV $\mathrm{R}^2$")